In [1]:
RAW_DATA = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\V4L_AMA-07008_THT3703-4_85C.csv'
out_path = r'C:\Users\411792\Desktop\LIFE_TEST_DATA\V4L_AMA-07008_THT3703-4_85C.xlsx'

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from openpyxl.chart import BarChart3D,Reference

In [3]:
df = pd.read_csv(RAW_DATA)
df = df.sort_values(['Magnet','Head'])
grouped_df = df.groupby(['Magnet','Head']).first()
df = pd.merge(df, grouped_df, on=['Magnet','Head'] , suffixes=('','_REF'))
df['TTF'] = df.groupby(['Magnet','Head'])['StressTimeInc'].cumsum()/60
df['PCT_CNG'] = (df['Ohms']/df['Ohms_REF'] - 1)*100
df['P/F Status'] = np.where(abs(df['PCT_CNG'])>=20,1,0)
df['P/F Status'] = df.groupby(['Magnet','Head'])['P/F Status'].cumsum()
df['P/F Status'] = np.where(df['Head']!=df['Head'].shift(),0,np.where(df['Head']!=df['Head'].shift(-1),1,np.where(df['P/F Status']>0,1,0)))
df['P/F Flag (10%)'] = np.where(df['Head']!=df['Head'].shift(),0,np.where(df['P/F Status']!=df['P/F Status'].shift(),1,0))
df['F/S'] = np.where((df['P/F Flag (10%)']==1)&(abs(df['PCT_CNG'])>=20),'F','S')
df['SBR'] = np.where(df['Magnet']<14,'THT3703','THT3704')
df = df[['HeadID','Date','MagnetTemp','Magnet','Head','Volts','mA','Carrier','StressTimeInc','StressVolts','StressOhms','TTF','Ohms','Ohms_REF','PCT_CNG','P/F Status','P/F Flag (10%)','F/S','SBR']]

dy = df.loc[df['P/F Flag (10%)'] == 1]
table = pd.pivot_table(dy, values='HeadID', index=['F/S','TTF'], columns=['SBR','StressVolts'], aggfunc='count', margins=False)

In [4]:
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, sheet_name='ALL', index=None)
table.to_excel(writer, sheet_name='SUM')
dy.to_excel(writer, sheet_name='TTF')
writer.save()